In [1]:
!pip install pandas

In [2]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium --yes
import folium

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [110]:
#Download store locations and remove unwanted data
df = pd.read_csv("https://raw.githubusercontent.com/ninjananjo/Clothing-Store-Competition/master/StoreLocations.csv")
print(df.shape)
df = df[['TOWN','NAME','ID','COUNTRY','Address1','Postcode','Host']]
#df = df[df['ID'].str.startswith("TS")]
df.head(10)

(8, 56)


,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host
0,LONDON (OXFORD CIRCUS),LONDON OXFORD CIRCUS,MS0325,ENG,"Ground Floor, 36-38 Great Castle Street",W1W 8LG,TS0001
1,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TM7329,ENG,214 Oxford Street,W1W 8LG,TS0001
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873
4,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE OUTFIT,OU0207,ENG,72-76,NE1 7DF,OU0207
5,Z USA LOS ANGELES,LOS ANGELES,TM8104,US,189 The Grove Drive,90036,TS1271
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012
7,HAYES,HAYES LOMBARDY RET PARK OUTFIT,OU0201,ENG,Unit 3,UB3 3EX,OU0201


In [111]:
#Group by Host
grouped_id = df.groupby('Host').apply(lambda x: ','.join(x.ID)).reset_index()
grouped_id.rename({0:'Branches'}, axis='columns', inplace=True)
df = df.groupby(['Host']).last()
df = df.merge(grouped_id, left_on='Host',right_on='Host')
df

,Host,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Branches
0,OU0201,HAYES,HAYES LOMBARDY RET PARK OUTFIT,OU0201,ENG,Unit 3,UB3 3EX,OU0201
1,OU0207,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE OUTFIT,OU0207,ENG,72-76,NE1 7DF,OU0207
2,TS0001,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,"MS0325,TM7329,TS0001"
3,TS0873,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873
4,TS1012,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012
5,TS1271,Z USA LOS ANGELES,LOS ANGELES,TM8104,US,189 The Grove Drive,90036,TM8104


In [112]:
#get latitude and longitude data
def getLocation(storeName, postcode, country):
    
    if country == 'ENG': country = "UK"
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(postcode+" "+country)
    location_alt = geolocator.geocode(storeName+" "+country)
    try:
        lat = location.latitude
        lng = location.longitude
    except:
        try:
            lat = location_alt.latitude
            lng = location_alt.longitude
        except:
            lat = 0
            lng = 0
            
    return(lat,lng)

In [113]:
latitude = []
longitude = []

for storeName, postcode, country in zip(df['NAME'],df['Postcode'],df['COUNTRY']):
    lati, lngi = getLocation(storeName, postcode, country)
    latitude.append(lati)
    longitude.append(lngi)

df.loc[:,'Latitude']= latitude
df.loc[:,'Longitude']= longitude
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
print(df.shape[0])
df

6


,Host,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Branches,Latitude,Longitude
0,OU0201,HAYES,HAYES LOMBARDY RET PARK OUTFIT,OU0201,ENG,Unit 3,UB3 3EX,OU0201,51.516683,-0.405498
1,OU0207,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE OUTFIT,OU0207,ENG,72-76,NE1 7DF,OU0207,54.975903,-1.612382
2,TS0001,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,"MS0325,TM7329,TS0001",51.515534,-0.141352
3,TS0873,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873,51.514098,-0.094914
4,TS1012,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012,53.462559,-2.346119
5,TS1271,Z USA LOS ANGELES,LOS ANGELES,TM8104,US,189 The Grove Drive,90036,TM8104,34.067233,-118.352108


In [114]:
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20190501' # Foursquare API version
LIMIT = 400 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
categoryId = '4bf58dd8d48988d1f2941735'

In [115]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    
    store_list = []
    lat_list = []
    lng_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        #Add region name 
        for i in range(0,resultsVen.shape[0]):
            store_list.append(name)
            lat_list.append(lat)
            lng_list.append(lng)

        store = pd.DataFrame([store_list,lat_list,lng_list]).T
        nearbyVenues = pd.concat([store, resultsVenCat], axis=1)

    return(nearbyVenues)

In [123]:
regions_venues = pd.DataFrame()

#request venue information for each region and store in regions_venues
for storeId in df['Host'].unique():
    store_info = df[df['Host']==storeId]
    regions_venues = regions_venues.append(getNearbyVenues(names=store_info['Host'],
                                                  latitudes=store_info['Latitude'],
                                                  longitudes=store_info['Longitude'],
                                                  radius=radius),ignore_index=True)
    
#rename columns
regions_venues.rename(columns={0:'Host ID',
                          1:'Host Lat',
                          2:'Host Lng',
                          'id':'Venue ID',
                          'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

print(regions_venues.shape[0])
regions_venues.head()

281


,Host ID,Host Lat,Host Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,OU0201,51.5167,-0.405498,52864beb11d2b469002d259b,Sports Direct,48,[United Kingdom],51.516690,-0.404791,Sporting Goods Shop
1,OU0201,51.5167,-0.405498,4de4bab352b1d38d29a363c3,Vitamin-shop,6177,"[197 Uxbridge Rd, Ealing, Greater London, W13...",51.511246,-0.316752,Sporting Goods Shop
2,OU0201,51.5167,-0.405498,556a095b498e0b3c18cc44af,The North Face,9521,"[Unit 16, Level 01 (Wembley Park Boulevard), W...",51.556978,-0.284201,Clothing Store
3,OU0201,51.5167,-0.405498,55cf1807498e04ea1d25b15e,Wembley Stadium Store,9847,"[Wembley Stadium, Wembley, Greater London, HA9...",51.557424,-0.279253,Sporting Goods Shop
4,OU0201,51.5167,-0.405498,52751b2c498e0d3d7f1843f1,Asics,9579,[Unit 5b Ground Floor (Wembley Park Boulevard)...,51.556914,-0.283201,Sporting Goods Shop


In [124]:
jdsports = regions_venues[regions_venues['Venue Name']=='JD Sports']
jdsports.to_csv('jdsports_proximty.csv')
jdsports.head()

,Host ID,Host Lat,Host Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
6,OU0201,51.5167,-0.405498,50ca2e02e4b0a678c04f68a5,JD Sports,9979,"[Unit 1B Stadium Retail Park (Wembley Pk Dr), ...",51.561356,-0.280536,Sporting Goods Shop
7,OU0201,51.5167,-0.405498,4baf53ccf964a5202ef83be3,JD Sports,5847,"[Units 221 222 & 205 The Chimes (High St), Uxb...",51.545247,-0.476358,Sporting Goods Shop
10,OU0201,51.5167,-0.405498,533fd0cd498ed8f545b0d8d0,JD Sports,7113,"[Unit 25 & 26 Broadway Centre, Ealing, Greater...",51.512847,-0.302992,Sporting Goods Shop
16,OU0201,51.5167,-0.405498,4ce698f5f1c6236ab06154f0,JD Sports,8639,"[St Anns Road, Harrow on the Hill, Greater Lon...",51.581607,-0.337121,Sporting Goods Shop
20,OU0201,51.5167,-0.405498,5a8ecf374ac28a25b17fd086,JD Sports,6197,"[Unit 28/29 Treaty Centre, Hounslow, Greater L...",51.467594,-0.363299,Sporting Goods Shop


In [125]:
df_map = jdsports
closest_jd = df_map.groupby(['Host ID'], as_index =False).min()
closest_jd

,Host ID,Host Lat,Host Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,OU0201,51.516683,-0.405498,4baf53ccf964a5202ef83be3,JD Sports,5847,"[173 High Street, Hounslow, Greater London, TW...",51.467594,-0.488720,Sporting Goods Shop
1,OU0207,54.975903,-1.612382,4bc09ecdb492d13aee80a460,JD Sports,25,[105A 106B B1-B7/9 The Gallery Metro Centre (R...,54.957119,-1.670618,Sporting Goods Shop
2,TS0001,51.515534,-0.141352,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.428388,-0.167446,Sporting Goods Shop
3,TS0873,51.514098,-0.094914,4ac518f1f964a520ecad20e3,JD Sports,3095,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
4,TS1012,53.462559,-2.346119,4b7ec528f964a52010fe2fe3,JD Sports,482,"[16-18 Arndale Centre, Barbarolli Mall, Manche...",53.379649,-2.348693,Sporting Goods Shop


In [128]:
m = folium.Map(location=[53.4626, -2.34612], zoom_start=6)

for branch, branch_lat, branch_lng, proximity in zip(closest_jd['Host ID'], closest_jd['Host Lat'], closest_jd['Host Lng'], closest_jd['Venue Distance']):
    if proximity < 2000:
        colour = 'green'
    else: 
        colour = 'red'
    folium.Marker(location=[branch_lat, branch_lng], popup=branch).add_to(m)
    folium.Circle(location=[branch_lat, branch_lng], radius=2000, fill=False, color=colour).add_to(m)

for branch_lat, branch_lng, jd_lat, jd_lng in zip(df_map['Host Lat'], df_map['Host Lng'], df_map['Venue Lat'], df_map['Venue Lng']):
    folium.Circle(location=[jd_lat, jd_lng], popup='JD Sports', radius=50, fill=True, color='black').add_to(m)

m

In [ ]:

#Add map feature with TM/TS as centre point and surrounding JD Sports.

In [131]:
#Stores without a JD Sports within a 2km radius
closest_jd[closest_jd['Venue Distance']>2000]

,Host ID,Host Lat,Host Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,OU0201,51.516683,-0.405498,4baf53ccf964a5202ef83be3,JD Sports,5847,"[173 High Street, Hounslow, Greater London, TW...",51.467594,-0.488720,Sporting Goods Shop
3,TS0873,51.514098,-0.094914,4ac518f1f964a520ecad20e3,JD Sports,3095,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
